In [23]:
#Make the necessary imports
import os
import sys
import numpy as np
import tarfile
import warnings
warnings.filterwarnings("ignore")
from zipfile import ZipFile
!pip install wget
import wget
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense, Input, GlobalMaxPooling1D
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Embedding, LSTM
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.initializers import Constant

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9656 sha256=6bb1367289e078b65f1fe6ff359d9cb8d3c7284e52a1723b2c6351a4ad00ad14
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget


In [24]:
%%capture

try:
  from google.colab import files
  !wget -P DATAPATH http://nlp.stanford.edu/data/glove.6B.zip
  !unzip DATAPATH/glove.6b.zip -d DATAPATH/glove.6B

  !wget -P DATAPATH http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
  !tar -xvf DATAPATH/aclImdb_v1.tar.gz -C DATAPATH

  BASE_DIR='DATAPATH'

except ModuleNotFoundError:

  if not os.path.exists('Data/glove.6B'):
    os.mkdir('Data/glove.6B')

    url='http://nlp.stanford.edu/data/glove.6B.zip'
    wget.download(url, 'Data')

    temp='Data/glove.6B.zip'
    file=ZipFile(temp)
    file.extractall('Data/glove.6B')
    file.close()

    if not os.path.exists('Data/aclImdb'):

        url='http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz'
        wget.download(url,'Data')

        temp='Data/aclImdb_v1.tar.gz'
        tar = tarfile.open(temp, "r:gz")
        tar.extractall('Data')
        tar.close()

    BASE_DIR = 'Data'

In [25]:
GLOVE_DIR=os.path.join(BASE_DIR, 'glove.6B')
TRAIN_DATA_DIR=os.path.join(BASE_DIR, 'aclImdb/train')
TEST_DATA_DIR = os.path.join(BASE_DIR, 'aclImdb/test')

In [26]:
MAX_SEQUENCE_LENGTH=1000
MAX_NUM_WORDS=20000
EMBEDDING_DIM=100
VALIDATION_SPLIT=0.2


In [30]:
#@ Loading and preprocessing:
def get_data(data_dir):
  texts=[]
  labels_index={'pos':1, 'neg':0}
  labels=[]
  for name in sorted(os.listdir(data_dir)):
    path=os.path.join(data_dir, name)
    if os.path.isdir(path):
      if name=='pos' or name=='neg':
        label_id=labels_index[name]
        for fname in sorted(os.listdir(path)):
          fpath=os.path.join(path, fname)
          text=open(fpath, encoding='utf8').read()
          texts.append(text)
          labels.append(label_id)

  return texts, labels

train_texts, train_labels=get_data(TRAIN_DATA_DIR)
test_texts, test_labels=get_data(TEST_DATA_DIR)
labels_index={'pos':1, 'neg':0}

#Just to see how the data looks like.
print(train_texts[0])
print(train_labels[0])
print(test_texts[24999])
print(test_labels[24999])

Story of a man who has unnatural feelings for a pig. Starts out with a opening scene that is a terrific example of absurd comedy. A formal orchestra audience is turned into an insane, violent mob by the crazy chantings of it's singers. Unfortunately it stays absurd the WHOLE time with no general narrative eventually making it just too off putting. Even those from the era should be turned off. The cryptic dialogue would make Shakespeare seem easy to a third grader. On a technical level it's better than you might think with some good cinematography by future great Vilmos Zsigmond. Future stars Sally Kirkland and Frederic Forrest can be seen briefly.
0
I've seen this story before but my kids haven't. Boy with troubled past joins military, faces his past, falls in love and becomes a man. The mentor this time is played perfectly by Kevin Costner; An ordinary man with common everyday problems who lives an extraordinary conviction, to save lives. After losing his team he takes a teaching posi